In [1]:
import requests 
import string
from bs4 import BeautifulSoup  
import logging  
logging.basicConfig(level=logging.DEBUG)

In [2]:
def get_all_movies():  
    """ returns all the movie urls from boxofficemojo.com in a list"""

    # Alphabet loop for how movies are indexed including
    # movies that start with a special character or number
    index = ["NUM"] + list(string.ascii_uppercase)

    # List of movie urls
    movies_list = []

    # Loop through the pages for each letter
    for letter in index:

        # Loop through the pages within each letter
        for num in range(1, 20):
            url = ("http://www.boxofficemojo.com/movies/alphabetical.htm?"
                   "letter=" + letter + "&page=" + str(num))
            try:
                response = requests.get(url)
                page = response.text
                soup = BeautifulSoup(page, "lxml")
                rows = soup.find(id="body").find("table").find("table").find_all(
                    "table")[1].find_all("tr")

                # skip index row
                if len(rows) > 1:
                    counter = 1
                    for row in rows:
                        # skip index row
                        if counter > 1:
                            link = row.td.font.a['href']
                            # don't add duplicates
                            if link not in movies_list:
                                movies_list.append(link)

                        counter += 1
            except (Exception, e):
                logging.exception(e)

    return movies_list

In [3]:
all_movies = get_all_movies()

In [4]:
all_movies_subset = all_movies[:100]

In [ ]:
#print(len(movie_chunks))
#print(len(all_movies))
#n = list(range(len(all_movies)+1))

In [ ]:
#L = int(len(all_movies)/80)
#intervals = n[::L]
#intervals.append(len(all_movies)-1)
#print (intervals)

In [ ]:
#print(intervals)

In [ ]:
#for i in intervals:
#    for movie in all_movies[intervals[i]:intervals[i+1]]:
#        print(movie)
    

In [ ]:
#L = int(len(all_movies)/40)
#L

In [97]:
#url = 'http://www.boxofficemojo.com/movies/?id=addamsfamily.htm'
url = 'http://www.boxofficemojo.com/movies/?id=casablanca.htm'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")


In [98]:
def get_movie_value(soup, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    if not obj: 
        return None
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text # RETURN TEXT OF NEXT SIBLING
    else:
        return None

In [99]:
def get_movie_value_next_next(soup, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    
    if obj:
        return obj.next.next.text
    else:
        return None

In [100]:
try:
    producer = get_movie_value_next(soup, 'Producer')
    producers = re.sub('([a-z()])([A-Z])', '\g<1>, \g<2>', producer).split(",") 
except TypeError:
    producers = float('NaN')
except AttributeError:
    producers = float('NaN')

In [101]:
try:
    director = get_movie_value_next(soup,'Director')
    director = re.sub('([a-z()])([A-Z])', '\g<1>, \g<2>', director).split(",") 
except TypeError:
    director = float('NaN')


In [102]:
# actors
try:
    actor = get_movie_value_next(soup,'Actor')
    v = re.sub('([a-z()])([A-Z])', '\g<1>,\g<2>', actor)
    actors = re.sub('[(*\']','', v)
    actors = actors.split(",") 
except TypeError:
    actors = float('NaN')

In [103]:
genre = get_movie_value(soup, 'Genre[^a-z]') 

In [104]:
rating = get_movie_value(soup,'MPAA Rating')


In [105]:
import dateutil.parser

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date
# release date 
try:
    raw_release_date = get_movie_value(soup,'Release Date')
    release_date = to_date(raw_release_date)
except AttributeError:
    release_date = float('NaN')

In [106]:
# title
title_string = soup.find('title').text
title = title_string.split('(')[0].strip()

In [107]:
# domestic gross
try:
    raw_domestic_total_gross = get_movie_value(soup,'Domestic Total')
    domestic_total_gross = money_to_int(raw_domestic_total_gross)
except AttributeError:
    domestic_total_gross = float('NaN')

In [108]:
# opening weekend
try:
    opening_weekend_gross = get_movie_value_next(soup,'Opening Weekend')
    opening_weekend_gross = money_to_int(opening_weekend_gross)
except AttributeError:
    opening_weekend_gross = float('NaN')

In [109]:
def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def money_to_int_2(moneystring):
    if moneystring != 'N/A':
        moneystring = moneystring.replace('$', '').replace('.','').replace(',','').replace(' million', '000000')
        return int(moneystring)

In [110]:
#production budget
try:
    production_budget = get_movie_value(soup, 'Production Budget')
    production_budget = money_to_int_2(production_budget)
except AttributeError:
    production_budget = float('NaN')

In [111]:
def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

In [112]:
#movie runtime
try:
    raw_runtime = get_movie_value(soup,'Runtime')
    runtime = runtime_to_minutes(raw_runtime)
except AttributeError:
    runtime = float('NaN')

In [113]:

def strip_and_return_int(s):
    t = s.replace('$', '').replace(',','')
    t = int(t)
    return t

In [114]:
try:
    widest_release = get_movie_value_next_next(soup, 'Widest')
    widest_release = strip_and_return_int(widest_release.split()[0])
except AttributeError:
    widest_release = float('NaN')

In [116]:
title, genre, runtime, rating, producers, director, actors, release_date, domestic_total_gross, opening_weekend_gross, production_budget,widest_release

('Casablanca',
 'Romance',
 102,
 'Unrated',
 ['Hal B. Wallis'],
 ['Michael Curtiz'],
 ['Humphrey Bogart',
  'Ingrid Bergman',
  'Paul Henreid',
  'Claude Rains',
  'Conrad Veidt',
  'Sydney Greenstreet',
  'Peter Lorre',
  'S.Z. Sakall'],
 datetime.datetime(1942, 11, 26, 0, 0),
 nan,
 nan,
 950000,
 nan)

In [33]:
import re

In [34]:
get_genres(soup)

['TV Adaptation (Live Action)']

In [30]:
genres = soup.find(id="body").find(text=re.compile("Genres"))

In [31]:
genres = genres.findParent().findNextSibling().find_all('tr')

AttributeError: 'NoneType' object has no attribute 'find_all'

In [1]:
def get_genres(soup):
    """ returns all genres from specific movie page at boxofficemojo.com"""
    genres_list = []
    try:
        genres = soup.find(id="body").find(text=re.compile("Genres"))
        genres = genres.findParent().findNextSibling().find_all('tr')
        genre_count = 0
        for genre in genres:
            if genre_count > 0:
                genres_list.append(genre.td.font.a.text)
            genre_count += 1
    except LookupError:
        try:
            genres = soup.find(id="body").find(text=re.compile("Genre"))
            genres = genres.findNextSibling().text
            genres_list.append(genres)
        except:
            genres_list.append("N/A")
    return genres_list


def get_title(soup):
    """returns title from specific movie page at boxofficemojo.com"""
    try:
        title = soup.find("title").text.rsplit('(', 1)[0].strip()
    except LookupError:
        title = "N/A"
    return title


def get_release_date(soup):
    """returns datetime value of release date from specific movie
    page at boxofficemojo.com
    """
    try:
        date = soup.find(id="body").find(text=re.compile("Release Date"))
        date = date.findNextSibling().text
        date = datetime.strptime(date, "%B %d, %Y")
        return date
    except LookupError:
        return "N/A"


def get_distributor(soup):
    """returns movie distributor from specific movie page at boxofficemojo.com"""
    try:
        distributor = soup.find(id="body").find(text=re.compile("Distributor"))
        distributor = distributor.findNextSibling().text
        return distributor
    except LookupError:
        return "N/A"


def get_rating(soup):
    """returns MPAA Rating from specific movie page at boxofficemojo.com"""
    try:
        rating = soup.find(id="body").find(text=re.compile("MPAA Rating"))
        rating = rating.findNextSibling().text
        return rating
    except LookupError:
        return "N/A"


def get_runtime(soup):
    """returns integer value of runtime from specific movie page at boxofficemojo.com"""
    try:
        runtime = soup.find(id="body").find(text=re.compile("Runtime"))
        runtime = runtime.findNextSibling().text
        time_splits = runtime.split("hrs.")
        try:
            hrs = int(time_splits[0]) * 60
        except LookupError:
            hrs = 0
        mins = int(time_splits[1].split(" min.")[0].strip())
        total = hrs + mins
        return total
    except LookupError:
        return "N/A"


def get_budget(soup):
    """returns movie budget from specific movie page at boxofficemojo.com"""
    try:
        budget = soup.find(id="body").find(text=re.compile("Production Budget"))
        budget = budget.findNextSibling().text
        if budget != "N/A":
            budget = int(budget.split("million")[0].split("$")[1].strip()) * 1000000
        return budget
    except LookupError:
        return "N/A"


def get_domestic_gross(soup):
    """returns integer value of domestic gross from specific movie page at boxofficemojo.com"""
    try:
        gross = soup.find(id="body").find(text=re.compile("Domestic Total Gross: "))
        gross = gross.findNextSibling().text
        gross = int(gross.replace("$", "").replace(",", ""))
        return gross
    except LookupError:
        try:
            gross = soup.find(id="body").find(tex=re.compile("Domestic:"))
            gross = gross.findParent().findNextSibling().text
            return gross
        except:
            return "N/A"
        
        
def get_movie_value_next(soup, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    
    if obj:
        return obj.next.text
    else:
        return None

def get_actors(soup):
    actor = get_movie_value_next(soup,'Actor')
    v = re.sub('([a-z()])([A-Z])', '\g<1>, \g<2>', actor)
    actors = re.sub('[(*\']','', v)
    actors = actors.split(",")
    return actors   

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def get_opening_weekend(soup):
    opening_weekend_gross = get_movie_value_next(soup,'Opening Weekend')
    opening_weekend_gross = money_to_int(opening_weekend_gross)
    return opening_weekend_gross
    
    
def scrape_movie_data(movie_list, start=0, end=20000):
    """returns dictionary of movies and relevant data from boxofficemojo.com:
    genres(as a list), release date, distributor, runtime, MPAA rating,
    budget, gross domestic revenue
    """
    movie_data_list = {}
    counter = 0
    for movie in movie_list:
        try:
            if start < counter < end and counter < len(movie_list):
                url = "http://www.boxofficemojo.com/" + movie
                
                response = requests.get(url)
                time.sleep(0.1)
                page = response.text
                soup = BeautifulSoup(page, "lxml")
                print(movie)
                movie_data_list[get_title(soup)] = [movie, get_genres(soup), get_release_date(soup), 
                                                    get_distributor(soup), get_runtime(soup), 
                                                    get_rating(soup), get_budget(soup), 
                                                    get_domestic_gross(soup), get_actors(soup), 
                                                    get_opening_weekend(soup)]
                
            counter += 1
        except: #(Exception, e):
            pass

    return movie_data_list


#def main():
#    pass

#if __name__ == "__main__":
#    main()

In [2]:
from datetime import datetime
import time

movie_data_subset = scrape_movie_data(all_movies_subset, start=0, end=20000)

NameError: name 'all_movies_subset' is not defined

In [16]:
movie_data_subset

{}

In [8]:
import pandas as pd
df = pd.DataFrame(movie_data_subset)

In [10]:
df = df.transpose()

In [ ]:
movies_list_A = []
url = 'http://www.boxofficemojo.com/movies/alphabetical.htm?letter=A&page=1'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')
rows = soup.find(id = 'body').find('table').find('table').find_all('table')[1].find_all("tr")
if len(rows) > 1:
    counter = 1
    for row in rows:
    # skip index row
        if counter > 1:
            link = row.td.font.a['href']
            # don't add duplicates
            if link not in movies_list:
                movies_list_A.append(link)

        counter += 1

In [ ]:
url = 'http://www.boxofficemojo.com/movies/?id=addamsfamily.htm'

response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")


In [ ]:
def get_title(soup):
    """returns title from specific movie page at boxofficemojo.com"""
    try:
        title = soup.find("title").text.rsplit('(', 1)[0].strip()
    except LookupError:
        title = "N/A"
    return title

In [ ]:
get_title(soup)

In [ ]:
import re

In [ ]:
soup.find(text = re.compile('Actor')).next.text

In [ ]:
def get_movie_value_next(soup, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    
    if obj:
        return obj.next.text
    else:
        return None

In [ ]:
def get_actors(soup):
    actor = get_movie_value_next(soup,'Actor')
    v = re.sub('([a-z()])([A-Z])', '\g<1>,\g<2>', actor)
    actors = re.sub('[(*\']','', v)
    actors = actors.split(",")
    return actors

In [ ]:
get_actors(soup)